In [1]:
import os
import sys
module_path_dont_know = os.path.abspath(os.path.join('../COBRAS_dont_know'))
module_path_testing = os.path.abspath(os.path.join('../COBRAS_testing'))

if module_path_dont_know not in sys.path:
    sys.path.append(module_path_dont_know)
    print("module path of dont_know added")

if module_path_testing not in sys.path:
    sys.path.append(module_path_testing)
    print("module path of testing added")
    
from before_clustering.generate_folds import generate_folds_for_dataset
from clustering_algorithms.kmeans_fixed_representative import KmeansFixedRepresentative
from clustering_algorithms.kmeans_plus_fixed_representative import KmeansPlusFixedRepresentative
from cobras.cobras import COBRAS
from cobras.cobras_logger import COBRASLogger
from cobras.querier.labelquerier import LabelQuerier
from cobras.querier.weak_querier import WeakQuerier
from cobras.super_instances.superinstance_select_representative import SuperInstance_select_representative_Builder
from config import FOLD_RESULT_DIR, FIGURE_DIR
from evaluate_clusterings.calculate_aligned_rank import calculate_and_write_aligned_rank
from evaluate_clusterings.calculate_aris import calculate_n_times_n_fold_aris_for_testnames
from evaluate_clusterings.calculate_average_aris import calculate_average_aris
from generate_clusterings.clustering_task import make_n_run_10_fold_cross_validation
from heuristics.select_super_instance_heuristics import *
from heuristics.splitlevel_estimation_strategy import *
from pathlib import Path
from present_results.intra_pred_predictions import plot_overall_predictions, plot_predictions_per_dataset
from present_results.plot_aligned_rank import plot_rank_comparison_file
from present_results.plot_aris import plot_average_ARI_per_dataset
from present_results.plot_aris import plot_overall_average_ARI
from present_results.plot_query_reuse import calculate_and_plot_query_reuse
from present_results.plot_query_reuse_per_dataset import calculate_and_plot_query_reuse_per_dataset
from run_locally.run_tests import run_clustering_tasks_locally
from util.datasets import Dataset

print("imports succeeded")

imports succeeded


In [2]:
if not Path(FOLD_RESULT_DIR).exists():
    generate_folds_for_dataset()
    print("done")

In [3]:
splitstrat = StandardSplitLevelEstimationStrategyAlwayskmeans(SelectMostInstancesHeuristic())
clusterer_1 = COBRAS()
#querier = LabelQuerier(None,100)
querier = WeakQuerier(None, None ,100,'local_nondet')
test_name_1 = "KEEP_COBRAS_10fold"
all_dataset_names = Dataset.get_standard_dataset_names()#["iris"]#
clustering_tasks = make_n_run_10_fold_cross_validation(test_name_1, clusterer_1, querier, all_dataset_names, 10)

In [4]:
clusterer_2 = COBRAS(select_next_query=True, intra_pred=True,max_dist=False)
test_name_2 = "KEEP_intra_pred_min_10fold"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_2, clusterer_2, querier, all_dataset_names, 10))

In [5]:
clusterer_3 = COBRAS(select_next_query=True, intra_pred=True,max_dist=True)
test_name_3 = "KEEP_intra_pred_max_10fold"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_3, clusterer_3, querier, all_dataset_names, 10))

In [6]:
clusterer_4 = COBRAS(select_next_query=True, max_dist=False)
test_name_4 = "KEEP_inter_pred_min_10fold"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_4, clusterer_4, querier, all_dataset_names, 10))

In [7]:
clusterer_5 = COBRAS(select_next_query=True, inter_max=True)
test_name_5 = "KEEP_inter_pred_super_max_10fold"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_5, clusterer_5, querier, all_dataset_names, 10))

In [8]:
clusterer_6 = COBRAS(select_next_query = True, posterior_inter_pred=True)
test_name_6 = "KEEP_posterior_inter_pred_10fold"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_6, clusterer_6, querier, all_dataset_names, 10))

In [9]:
run_clustering_tasks_locally(clustering_tasks,4)
print("done")

0it [00:00, ?it/s]

done


In [10]:
test_names = [test_name_1, test_name_2, test_name_3, test_name_4, test_name_5, test_name_6]
comparison_name = 'FINAL_extra_queries'
nb_of_cores = 4
query_budget = 100

#print(clusterer_1.logger.nr_reused_constraints)

calculate_n_times_n_fold_aris_for_testnames(test_names, nb_cores=nb_of_cores)
calculate_average_aris(test_names, query_budget)
calculate_and_write_aligned_rank(test_names,comparison_name)

plot_average_ARI_per_dataset(comparison_name, test_names, test_names)
plot_overall_average_ARI(comparison_name, test_names, test_names)
plot_rank_comparison_file(comparison_name, test_names, test_names)

#plot_predictions_per_dataset(comparison_name, test_names)
#plot_overall_predictions(comparison_name, test_names)

#calculate_and_plot_query_reuse(comparison_name, test_names)
#calculate_and_plot_query_reuse_per_dataset(comparison_name, test_names)

print(f"all resulting plots are in {Path(FIGURE_DIR)/comparison_name}")

  0%|          | 0/6 [00:00<?, ?it/s]

Calculating ARIs for n-times n-fold:  ['KEEP_COBRAS_10fold', 'KEEP_intra_pred_min_10fold', 'KEEP_intra_pred_max_10fold', 'KEEP_inter_pred_min_10fold', 'KEEP_inter_pred_super_max_10fold', 'KEEP_posterior_inter_pred_10fold']
already calculated
Calculating average ARIs


100%|██████████| 6/6 [01:45<00:00, 17.53s/it]


calculating average rank
all resulting plots are in \Users\nicol\Documents\KUL 2020-2021\thesis\code\results\results\figures\FINAL_extra_queries
